# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f615cb3a2b0>
├── 'a' --> tensor([[-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808]])
└── 'x' --> <FastTreeValue 0x7f6180f0f970>
    └── 'c' --> tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                        [ 0.9687, -0.8123,  0.4566, -1.1357],
                        [ 2.0467, -1.2120,  0.5499,  0.9780]])

In [4]:
t.a

tensor([[-0.3596,  2.3718, -1.5135],
        [-0.6219, -0.4696,  0.3808]])

In [5]:
%timeit t.a

70.8 ns ± 0.00581 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f615cb3a2b0>
├── 'a' --> tensor([[-0.3381,  0.3223,  0.5112],
│                   [ 0.0972, -0.4681, -1.3466]])
└── 'x' --> <FastTreeValue 0x7f6180f0f970>
    └── 'c' --> tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                        [ 0.9687, -0.8123,  0.4566, -1.1357],
                        [ 2.0467, -1.2120,  0.5499,  0.9780]])

In [7]:
%timeit t.a = new_value

67.2 ns ± 0.0313 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808]]),
    x: Batch(
           c: tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                      [ 0.9687, -0.8123,  0.4566, -1.1357],
                      [ 2.0467, -1.2120,  0.5499,  0.9780]]),
       ),
)

In [10]:
b.a

tensor([[-0.3596,  2.3718, -1.5135],
        [-0.6219, -0.4696,  0.3808]])

In [11]:
%timeit b.a

59.2 ns ± 0.0109 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5862,  0.8694, -1.6671],
               [ 0.5105,  0.1423, -1.6231]]),
    x: Batch(
           c: tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                      [ 0.9687, -0.8123,  0.4566, -1.1357],
                      [ 2.0467, -1.2120,  0.5499,  0.9780]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.0558 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

825 ns ± 0.259 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 567 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 951 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f60c910bd30>
├── 'a' --> tensor([[[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]],
│           
│                   [[-0.3596,  2.3718, -1.5135],
│                    [-0.6219, -0.4696,  0.3808]]])
└── 'x' --> <FastTreeValue 0x7f60c910bcd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 31.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f60c96d9b20>
├── 'a' --> tensor([[-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808],
│                   [-0.3596,  2.3718, -1.5135],
│                   [-0.6219, -0.4696,  0.3808]])
└── 'x' --> <FastTreeValue 0x7f6180e950d0>
    └── 'c' --> tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                        [ 0.9687, -0.8123,  0.4566, -1.1357],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]],
       
               [[-0.3596,  2.3718, -1.5135],
                [-0.6219, -0.4696,  0.3808]]]),
    x: Batch(
           c: tensor([[[ 0.1987, -2.2553,  0.5566,  0.8953],
                       [ 0.9687, -0.8123,  0.4566, -1.1357],
                       [ 2.0467, -1.2120,  0.5499,  0.9780]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808],
               [-0.3596,  2.3718, -1.5135],
               [-0.6219, -0.4696,  0.3808]]),
    x: Batch(
           c: tensor([[ 0.1987, -2.2553,  0.5566,  0.8953],
                      [ 0.9687, -0.8123,  0.4566, -1.1357],
                      [ 2.0467, -1.2120,  0.5499,  0.9780],
                      [ 0.1987, -2.2553,  0.5566,  0.8953],
                      [ 0.9687, -

In [28]:
%timeit Batch.cat(batches)

161 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
